In [1]:
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer
from skopt.utils import use_named_args
from skimage.feature import local_binary_pattern
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import csv
from skopt import gp_minimize
import cv2
import concurrent.futures
from os import listdir
from os.path import isfile, join
import numpy as np

path = '../data'
categorias = ['NORMAL', 'PNEUMONIA']

# Definir el espacio de búsqueda para los hiperparámetros del LBP
espacio_lbp = [
    Integer(1, 5, name='radio'),
    Integer(8, 24, name='n_puntos')  # Ajustado para LBP
]

def procesar_imagen(imagen_path, lbp_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    lbp = local_binary_pattern(imagen, lbp_params['n_puntos'], lbp_params['radio'], method='uniform')
    # Calcular el histograma de LBP
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_params['n_puntos'] + 3), range=(0, lbp_params['n_puntos'] + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)

    return hist, etiqueta

def cargar_datos(ruta, lbp_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, lbp_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    return np.array(datos), np.array(etiquetas)

nombre_archivo_csv = "../results/lbp_combination_knn.csv"
with open(nombre_archivo_csv, 'w', newline='') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['Radio', 'Número de Puntos', 'Accuracy', 'Recall', 'Specificity', 'Precision', 'F1-Score', 'AUC'])

@use_named_args(espacio_lbp)
def funcion_objetivo_lbp(radio, n_puntos):
    lbp_params = {'radio': radio, 'n_puntos': n_puntos}
    datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), lbp_params)
    datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), lbp_params)
    modelo = KNeighborsClassifier(n_jobs=-1)
    modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
    etiquetas_predichas = modelo.predict(datos_prueba)
    accuracy = accuracy_score(etiquetas_prueba, etiquetas_predichas)
    f1 = f1_score(etiquetas_prueba, etiquetas_predichas)
    precision = precision_score(etiquetas_prueba, etiquetas_predichas)
    recall = recall_score(etiquetas_prueba, etiquetas_predichas)
    cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    auc = roc_auc_score(etiquetas_prueba, modelo.predict_proba(datos_prueba)[:, 1])
    
    with open(nombre_archivo_csv, 'a', newline='') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([radio, n_puntos, accuracy, recall, specificity, precision, f1, auc])
    
    return -f1

resultado = gp_minimize(funcion_objetivo_lbp, espacio_lbp, n_calls=20, random_state=42, n_jobs=-1, verbose=True)

print(f"Mejor F1-Score: {-resultado.fun}")
print(f"Mejores hiperparámetros: Radio={resultado.x[0]}, Número de Puntos={resultado.x[1]}")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 56.0166
Function value obtained: -0.8150
Current minimum: -0.8150
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 67.2075
Function value obtained: -0.8049
Current minimum: -0.8150
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 52.1892
Function value obtained: -0.8110
Current minimum: -0.8150
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 56.9332
Function value obtained: -0.8203
Current minimum: -0.8203
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 63.2350
Function value obtained: -0.8119
Current minimum: -0.8203
Iteration No: 6 star

c:\Users\Anabel\AppData\Local\Programs\Python\Python312\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [5, 8] before, using random point [4, 10]
  warnings.warn(


Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 56.7419
Function value obtained: -0.8281
Current minimum: -0.8401
Iteration No: 13 started. Searching for the next optimal point.


c:\Users\Anabel\AppData\Local\Programs\Python\Python312\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [5, 8] before, using random point [3, 17]
  warnings.warn(


Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 59.3101
Function value obtained: -0.8131
Current minimum: -0.8401
Iteration No: 14 started. Searching for the next optimal point.


c:\Users\Anabel\AppData\Local\Programs\Python\Python312\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [5, 8] before, using random point [2, 17]
  warnings.warn(


Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 61.2170
Function value obtained: -0.8234
Current minimum: -0.8401
Iteration No: 15 started. Searching for the next optimal point.


c:\Users\Anabel\AppData\Local\Programs\Python\Python312\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [5, 8] before, using random point [3, 17]
  warnings.warn(


Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 52.1051
Function value obtained: -0.8131
Current minimum: -0.8401
Iteration No: 16 started. Searching for the next optimal point.
Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 49.8173
Function value obtained: -0.8413
Current minimum: -0.8413
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 57.7192
Function value obtained: -0.8271
Current minimum: -0.8413
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 52.2315
Function value obtained: -0.8214
Current minimum: -0.8413
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 37.2372
Function value obtained: -0.8185
Current minimum: -0.8413
Iteration No: 